# SDoH prediction with multi-modal BERT


## Setup

We'll need [the Transformers library](https://huggingface.co/transformers/) by Hugging Face:

In [30]:

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from tqdm.notebook import tqdm

#updated imports for multimodal input 

from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import BertWithTabular#AutoModelWithTabular

#done

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

## Data Exploration


In [31]:
#set data path
va_path = "/home/gridsan/stynan/sdoh_model/sdoh_dataset_processed.xlsx" 

df = pd.read_excel(va_path)

In [32]:
#make list of texts and labels 
df = df[df["edu_level_composite"] != -1] #remove all rows that have missing values or unknowns in edu level 
df.edu_level_composite -= 1
df = df[df["open_response"].notna()]
df.fillna(-1, inplace = True)

df = df.iloc[1:] #removing the first row since it is just not necessary

In [33]:
df.shape

(6834, 107)

In [34]:
PRE_TRAINED_MODEL_NAME = '/home/gridsan/stynan/sdoh_model/bert-base-uncased'

In [35]:
@dataclass
class ModelArguments:
  """
  Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  """

  model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
  config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
  tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
  cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )

    
@dataclass
class MultimodalDataTrainingArguments:
  """
  Arguments pertaining to how we combine tabular features
  Using `HfArgumentParser` we can turn this class
  into argparse arguments to be able to specify them on
  the command line.
  """

  data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
  column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
  })

  column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
  })

  categorical_encode_type: str = field(default='ohe',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            'choices': ['ohe', 'binary', 'label', 'none']
                                        })

  numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
  task: str = field(default="classification",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

  mlp_division: int = field(default=4,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
  combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
  mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
  numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
  use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
  mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
  gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

  def __post_init__(self):
      assert self.column_info != self.column_info_path
      if self.column_info is None and self.column_info_path:
          with open(self.column_info_path, 'r') as f:
              self.column_info = json.load(f)

The tokenizer is doing most of the heavy lifting for us. We also return the review texts, so it'll be easier to evaluate the predictions from our model. Let's split the data:

In [44]:
text_cols = ["open_response"]

cat_cols = ['site_x',
            'gs_text34_x',
            'g1_05', 
            'gs_comorbid1',
            'gs_comorbid2', 
            'g1_06m', 
            'g1_06y', 
            'g1_07a', 
            'g2_01',
            'g4_02',
            'g4_03a',
            'g4_05',
            'g4_08',
            'g5_05',
            'a1_01_2','a1_01_3','a1_01_4','a1_01_5','a1_01_6','a1_01_7','a1_01_8','a1_01_9','a1_01_10','a1_01_11','a1_01_12','a1_01_13','a1_01_14',
            'a3_10',
            'a3_17',
            'a3_18',
            'a4_01',
            'a4_02_1',
            'a4_02_2',
            'a4_02_3',
            'a4_02_4',
            'a4_02_5a',
             'a4_02_5a',
           'a4_02_5b',
            'a4_03',
            'a4_04',
            'a4_05',
           'a4_05',
            'a4_06',
            'a5_02',
            'a5_03',
            'a6_01',
            'a6_02_1',
            'a6_02_2',
            'a6_02_3',
            'a6_02_4',
            'a6_02_5',
            'a6_02_6',
            'a6_02_7',
            'a6_02_8',
            'a6_02_9',
            'a6_02_10',
            'a6_02_11',	
            'a6_02_12a',
            'a6_02_13',	
            'a6_02_14',	
            'a6_02_15',
            'a6_04',
            'a6_05',
            'a6_09',
            'a6_10'
           ]
numerical_cols = ['age_years', 'total_#_hospitals', 'Interview Attemps', 'g4_04', 'g4_07', 'a2_01','a3_11','a5_04' ]

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': "edu_level_composite",
    'label_list': [1,2,3,4]
}


model_args = ModelArguments(
    model_name_or_path='/home/gridsan/stynan/sdoh_model/bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='/home/gridsan/stynan/sdoh_model/',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='classification'
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict = True,
    per_device_train_batch_size=16,
    num_train_epochs=20,
    evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250
)

set_seed(training_args.seed)

In [45]:
#tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = BertTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
    model_max_length=512
)

Specified tokenizer:  /home/gridsan/stynan/sdoh_model/bert-base-uncased


In [46]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df, test_size=0.5, random_state=RANDOM_SEED)

In [47]:
df_train.to_csv('train.csv')
df_val.to_csv('val.csv')
df_test.to_csv('test.csv')

In [48]:
# Get Datasets
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

/home/gridsan/stynan/.local/lib/python3.8/site-packages/multimodal_transformers/data/data_utils.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = df[num_cols].fillna(df[num_cols].median())
/home/gridsan/stynan/.local/lib/python3.8/site-packages/multimodal_transformers/data/data_utils.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = df[num_cols].fillna(df[num_cols].median())
/home/gridsan/stynan/.local/lib/python3.8/site-packages/multimodal_transformers/data/data_utils.

In [50]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
num_labels = 4
tabular_config = TabularConfig(num_labels=num_labels,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

## Classification with BERT and Hugging Face

In [51]:

model = BertWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

model = model.to(device)

Some weights of the model checkpoint at /home/gridsan/stynan/sdoh_model/bert-base-uncased were not used when initializing BertWithTabular: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithTabular were not initialized from the model checkpoint at /home/gridsan/stynan/sd

We can use all of this knowledge to create a classifier that uses the BERT model:

In [52]:
from scipy.special import softmax
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
)

def calc_classification_metrics(p: EvalPrediction):
  pred_labels = np.argmax(p.predictions, axis=1)
  pred_scores = softmax(p.predictions, axis=1)[:, 1]
  labels = p.label_ids
  if len(np.unique(labels)) == 2:  # binary classification
      roc_auc_pred_score = roc_auc_score(labels, pred_scores)
      precisions, recalls, thresholds = precision_recall_curve(labels,
                                                                pred_scores)
      fscore = (2 * precisions * recalls) / (precisions + recalls)
      fscore[np.isnan(fscore)] = 0
      ix = np.argmax(fscore)
      threshold = thresholds[ix].item()
      pr_auc = auc(recalls, precisions)
      tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
      result = {'roc_auc': roc_auc_pred_score,
                'threshold': threshold,
                'pr_auc': pr_auc,
                'recall': recalls[ix].item(),
                'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
                'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
                }
  else:
      acc = (pred_labels == labels).mean()
      f1 = f1_score(y_true=labels, y_pred=pred_labels, average = 'weighted')
      result = {
          'precidsion'
          "acc": acc,
          "f1": f1,
          "acc_and_f1": (acc + f1) / 2,
          "mcc": matthews_corrcoef(labels, pred_labels)
      }

  return result

### Training

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #test_dataset = test_dataset,
    compute_metrics=calc_classification_metrics,
)

In [54]:
%%time
trainer.train()

/state/partition1/llgrid/pkg/anaconda/anaconda3-2021a/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.292910614013672, 'learning_rate': 4.967616580310881e-05, 'epoch': 0.12953367875647667, 'step': 25}
{'loss': 1.2415359497070313, 'learning_rate': 4.9352331606217614e-05, 'epoch': 0.25906735751295334, 'step': 50}
{'loss': 1.1841990661621093, 'learning_rate': 4.902849740932643e-05, 'epoch': 0.38860103626943004, 'step': 75}
{'loss': 1.212017822265625, 'learning_rate': 4.870466321243523e-05, 'epoch': 0.5181347150259067, 'step': 100}
{'loss': 1.1837298583984375, 'learning_rate': 4.8380829015544046e-05, 'epoch': 0.6476683937823834, 'step': 125}
{'loss': 1.1828997802734376, 'learning_rate': 4.805699481865285e-05, 'epoch': 0.7772020725388601, 'step': 150}
{'loss': 1.1289813232421875, 'learning_rate': 4.773316062176166e-05, 'epoch': 0.9067357512953368, 'step': 175}



{'loss': 1.0936590576171874, 'learning_rate': 4.740932642487047e-05, 'epoch': 1.0362694300518134, 'step': 200}
{'loss': 1.1113958740234375, 'learning_rate': 4.708549222797928e-05, 'epoch': 1.16580310880829, 'step': 225}
{'loss': 1.155003662109375, 'learning_rate': 4.676165803108808e-05, 'epoch': 1.2953367875647668, 'step': 250}



{'eval_loss': 1.0588400294845621, 'eval_acc': 0.5127304653204565, 'eval_f1': 0.41148551066325006, 'eval_acc_and_f1': 0.46210798799185326, 'eval_mcc': 0.23629790787361776, 'epoch': 1.2953367875647668, 'step': 250}
{'loss': 1.072750244140625, 'learning_rate': 4.643782383419689e-05, 'epoch': 1.4248704663212435, 'step': 275}
{'loss': 1.103773193359375, 'learning_rate': 4.61139896373057e-05, 'epoch': 1.5544041450777202, 'step': 300}
{'loss': 1.06382568359375, 'learning_rate': 4.5790155440414514e-05, 'epoch': 1.6839378238341969, 'step': 325}
{'loss': 1.092694091796875, 'learning_rate': 4.546632124352332e-05, 'epoch': 1.8134715025906736, 'step': 350}
{'loss': 1.105372314453125, 'learning_rate': 4.5142487046632126e-05, 'epoch': 1.9430051813471503, 'step': 375}



{'loss': 1.012935791015625, 'learning_rate': 4.481865284974093e-05, 'epoch': 2.0725388601036268, 'step': 400}
{'loss': 1.046302490234375, 'learning_rate': 4.4494818652849745e-05, 'epoch': 2.2020725388601035, 'step': 425}
{'loss': 1.030330810546875, 'learning_rate': 4.417098445595855e-05, 'epoch': 2.33160621761658, 'step': 450}
{'loss': 0.976514892578125, 'learning_rate': 4.384715025906736e-05, 'epoch': 2.461139896373057, 'step': 475}
{'loss': 0.97410400390625, 'learning_rate': 4.352331606217617e-05, 'epoch': 2.5906735751295336, 'step': 500}


...

{'loss': 0.101064453125, 'learning_rate': 4.9870466321243525e-06, 'epoch': 18.00518134715026, 'step': 3475}
{'loss': 0.0868603515625, 'learning_rate': 4.663212435233161e-06, 'epoch': 18.134715025906736, 'step': 3500}



{'eval_loss': 0.08178338511537064, 'eval_acc': 0.971027216856892, 'eval_f1': 0.9710071521734951, 'eval_acc_and_f1': 0.9710171845151936, 'eval_mcc': 0.958109425351346, 'epoch': 18.134715025906736, 'step': 3500}
{'loss': 0.1038818359375, 'learning_rate': 4.33937823834197e-06, 'epoch': 18.264248704663213, 'step': 3525}
{'loss': 0.0922705078125, 'learning_rate': 4.015544041450777e-06, 'epoch': 18.39378238341969, 'step': 3550}
{'loss': 0.09341796875, 'learning_rate': 3.6917098445595854e-06, 'epoch': 18.523316062176164, 'step': 3575}
{'loss': 0.082939453125, 'learning_rate': 3.367875647668394e-06, 'epoch': 18.65284974093264, 'step': 3600}
{'loss': 0.10423828125, 'learning_rate': 3.044041450777202e-06, 'epoch': 18.78238341968912, 'step': 3625}
{'loss': 0.08978515625, 'learning_rate': 2.7202072538860106e-06, 'epoch': 18.911917098445596, 'step': 3650}



{'loss': 0.1018359375, 'learning_rate': 2.3963730569948187e-06, 'epoch': 19.041450777202073, 'step': 3675}
{'loss': 0.060986328125, 'learning_rate': 2.0725388601036273e-06, 'epoch': 19.17098445595855, 'step': 3700}
{'loss': 0.0896337890625, 'learning_rate': 1.7487046632124352e-06, 'epoch': 19.300518134715027, 'step': 3725}
{'loss': 0.090400390625, 'learning_rate': 1.4248704663212437e-06, 'epoch': 19.430051813471504, 'step': 3750}



{'eval_loss': 0.07844798672205136, 'eval_acc': 0.9721978343576236, 'eval_f1': 0.9721372771557468, 'eval_acc_and_f1': 0.9721675557566852, 'eval_mcc': 0.9598868526662633, 'epoch': 19.430051813471504, 'step': 3750}
{'loss': 0.0834716796875, 'learning_rate': 1.1010362694300518e-06, 'epoch': 19.559585492227978, 'step': 3775}
{'loss': 0.08548828125, 'learning_rate': 7.772020725388602e-07, 'epoch': 19.689119170984455, 'step': 3800}
{'loss': 0.1129638671875, 'learning_rate': 4.533678756476684e-07, 'epoch': 19.818652849740932, 'step': 3825}
{'loss': 0.0868701171875, 'learning_rate': 1.295336787564767e-07, 'epoch': 19.94818652849741, 'step': 3850}


CPU times: user 1h 10min 6s, sys: 26min 24s, total: 1h 36min 31s
Wall time: 57min 15s


TrainOutput(global_step=3860, training_loss=0.3605542118685233)

In [55]:
torch.save(model.state_dict(), 'best_multimodal_model_state.bin')
model.eval()

BertWithTabular(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

## Evaluation

So how good is our model? Let's start by calculating the accuracy on the test data:

In [57]:
%%time
output = trainer.predict(test_dataset)

/state/partition1/llgrid/pkg/anaconda/anaconda3-2021a/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



CPU times: user 29 s, sys: 8.34 s, total: 37.4 s
Wall time: 34.8 s


In [61]:
test_dataset.labels

array([2, 0, 2, ..., 0, 0, 2])

In [59]:
best_predictions = output.predictions

In [63]:
best_predictions

array([[ 4.2196665 , -4.417337  ,  0.21525875,  1.3477846 ],
       [ 5.4384613 ,  1.0589525 , -3.375457  , -3.4238255 ],
       [ 2.7913105 , -4.1859922 ,  5.3319745 , -3.644537  ],
       ...,
       [ 8.368958  , -2.649276  , -2.1181278 , -2.9907756 ],
       [ 8.224091  , -3.5231922 , -1.4661903 , -2.4355128 ],
       [ 2.11276   , -2.4767573 ,  2.5345497 , -1.3108507 ]],
      dtype=float32)

In [64]:
print(classification_report(test_dataset.labels, np.argmax(output.predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.85      0.92      0.88      1501
           1       0.86      0.86      0.86       760
           2       0.85      0.76      0.81       693
           3       0.88      0.82      0.85       463

    accuracy                           0.86      3417
   macro avg       0.86      0.84      0.85      3417
weighted avg       0.86      0.86      0.86      3417



This is similar to the evaluation function, except that we're storing the text of the reviews and the predicted probabilities (by applying the softmax on the model outputs):

## References

- [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)
- [L11 Language Models - Alec Radford (OpenAI)](https://www.youtube.com/watch?v=BnpB3GrpsfM)
- [The Illustrated BERT, ELMo, and co.](https://jalammar.github.io/illustrated-bert/)
- [BERT Fine-Tuning Tutorial with PyTorch](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)
- [How to Fine-Tune BERT for Text Classification?](https://arxiv.org/pdf/1905.05583.pdf)
- [Huggingface Transformers](https://huggingface.co/transformers/)
- [BERT Explained: State of the art language model for NLP](https://towardsdatascience.com/bert-explained-state-of-the-art-language-model-for-nlp-f8b21a9b6270)